In [ ]:
import json

### Read Foldseek json file

In [ ]:
FS_="Foldseek_2024_09_14_20_03_36.json"
# filtering criteria for metrics TM-align only
TM_cut = 0.9
# print header
print("%20s %50s %4s %4s %4s %3s" % ("PDB", "Description", "Prob", "SeqID", "TM-s", "Score"))
print()
with open(FS_) as f:
    # read line
    d = json.load(f)
    # alignments
    ali = d[0]['results'][0]['alignments']
    # loop over keys
    for key in ali:
        # PDB name
        pdb = ali[key][0]['target']
        # sequence identify
        seqi = ali[key][0]['seqId']
        # description
        de = ali[key][0]['description'][:50]
        # TM score
        TM = float(ali[key][0]['eval'])
        if(TM<TM_cut): continue
        # probability
        p = float(ali[key][0]['prob'])
        # score
        score = float(ali[key][0]['score'])
        # print stuff
        print("%20s %50s %4.3lf %4.1lf %4.3lf %3.1lf" % (pdb, de, p, seqi, TM , score))